In [1]:
from keras.models import Sequential
from keras.layers import SimpleRNN, LSTM, Dense, Embedding
from keras.preprocessing import sequence
from keras.datasets import imdb

2024-07-22 20:13:31.882141: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-07-22 20:13:31.885604: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-07-22 20:13:31.895145: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-22 20:13:31.910917: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-22 20:13:31.915523: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-07-22 20:13:31.927606: I tensorflow/core/platform/cpu_feature_gu

# Creating a simple model with no feature engineering. 

Just for inference and to create an API

In [2]:
max_len = 100
max_features = 10000
batch_size = 32

In [3]:
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)
print(len(x_train), "Train data")
print(len(x_test), "Test data")

# Padding data.
x_train = sequence.pad_sequences(x_train, maxlen=max_len)
x_test = sequence.pad_sequences(x_test, maxlen=max_len)
print("train shape:", x_train.shape)
print("test shape:", x_test.shape)

25000 Train data
25000 Test data
train shape: (25000, 100)
test shape: (25000, 100)


In [4]:
model = Sequential()
model.add(Embedding(max_features, 200))
model.add(SimpleRNN(200, dropout=0.2, activation='relu'))
model.add(Dense(1, activation="sigmoid"))

model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

I0000 00:00:1721659416.330509  202746 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-07-22 20:13:36.335460: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2343] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [5]:
model.fit(x_train, y_train, batch_size=batch_size, epochs=15, validation_data=(x_test, y_test))

Epoch 1/15
782/782 ━━━━━━━━━━━━━━━━━━━━ 24s 29ms/step - accuracy: 0.5971 - loss: 0.6934 - val_accuracy: 0.6768 - val_loss: 0.5651
Epoch 2/15
782/782 ━━━━━━━━━━━━━━━━━━━━ 23s 29ms/step - accuracy: 0.7307 - loss: 0.5572 - val_accuracy: 0.6603 - val_loss: 0.5930
Epoch 3/15
782/782 ━━━━━━━━━━━━━━━━━━━━ 23s 29ms/step - accuracy: 0.7889 - loss: 0.4513 - val_accuracy: 0.7922 - val_loss: 0.4673
Epoch 4/15
782/782 ━━━━━━━━━━━━━━━━━━━━ 23s 29ms/step - accuracy: 0.8697 - loss: 0.3037 - val_accuracy: 0.8006 - val_loss: 0.4689
Epoch 5/15
782/782 ━━━━━━━━━━━━━━━━━━━━ 23s 29ms/step - accuracy: 0.9096 - loss: 0.2247 - val_accuracy: 0.7763 - val_loss: 0.5607
Epoch 6/15
782/782 ━━━━━━━━━━━━━━━━━━━━ 23s 29ms/step - accuracy: 0.9360 - loss: 0.1692 - val_accuracy: 0.7866 - val_loss: 0.5969
Epoch 7/15
782/782 ━━━━━━━━━━━━━━━━━━━━ 23s 30ms/step - accuracy: 0.9424 - loss: 0.1546 - val_accuracy: 0.7564 - val_loss: 0.7056
Epoch 8/15
782/782 ━━━━━━━━━━━━━━━━━━━━ 24s 30ms/step - accuracy: 0.9563 - loss: 0.1201 - 

In [6]:
score, acc = model.evaluate(x_test, y_test, batch_size=batch_size)
print("Test score:", score)
print("Test accuracy:", acc)

782/782 ━━━━━━━━━━━━━━━━━━━━ 4s 6ms/step - accuracy: 0.7939 - loss: 1.0245
Test score: 0.9872850775718689
Test accuracy: 0.7973200082778931


In [8]:
if acc > 0.85:
    model.save("sentiment-model.h5")
    print("model saved")

# Inference

In [2]:
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.datasets import imdb
from tensorflow.keras.models import load_model
import numpy as np

# Parameters
max_len = 100
max_features = 10000

word_index = imdb.get_word_index()

def preprocess_review(review, word_index, max_len):
    tokenizer = Tokenizer(num_words=max_features)
    tokenizer.word_index = word_index
    sequences = tokenizer.texts_to_sequences([review])
    
    padded_sequence = sequence.pad_sequences(sequences, maxlen=max_len)
    return padded_sequence

2024-07-23 10:19:25.160285: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-07-23 10:19:25.222676: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-07-23 10:19:25.284162: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-23 10:19:25.341448: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-23 10:19:25.357376: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-07-23 10:19:25.447359: I tensorflow/core/platform/cpu_feature_gu

1641221/1641221 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [6]:
model = load_model("sentiment-model.h5")

new_review = "The plot good and the characters were interesting."
preprocessed_review = preprocess_review(new_review, word_index, max_len)
prediction = model.predict(preprocessed_review)
predicted_label = (prediction > 0.5).astype("int32")
print(f"Predicted sentiment: {'positive' if predicted_label[0][0] == 1 else 'negative'}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 116ms/step
Predicted sentiment: positive


Delete it


In [7]:
# from fastapi import FastAPI
# from pydantic import BaseModel
# from tensorflow.keras.preprocessing import sequence
# from tensorflow.keras.preprocessing.text import Tokenizer
# from tensorflow.keras.datasets import imdb
# from tensorflow.keras.models import load_model


# #Parameters
# max_len = 100
# max_features = 10000

# word_to_index = imdb.get_word_index()

# model = load_model("sentiment-model.h5")

# def preprocess_review(review, word_to_index, max_len, max_features):
#     tokenizer = Tokenizer(num_words=max_features)
#     tokenizer.word_index = word_to_index
#     sequence = tokenizer.texts_to_sequence([review])
#     padded_sequence = sequence.pad_sequences(sequences, maxlen=max_len)
#     return padded_sequence

# app = FastAPI(title="IMDB Sentiment classifier API",
#               version="0.1")

# class Review(BaseModel):
#     text:str

# # @app.post("/predict")
# def predict_sentiment(review: str):
#     preprocessed_review = preprocess_review(review.txt, word_to_index, max_len, max_features)

#     prediction = model.predict(preprocessed_review)
#     predicted_label = (prediction > 0.5).astype("int32")

#     sentiment = 'positive' if predicted_label[0][0] == 1 else 'negative'

#     return {"Sentiment": sentiment, "Probability": prediction}